In [2]:
import os, glob, requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd

bs4.__version__

'4.11.1'


 
[https://grobid.readthedocs.io/en/latest/Grobid-docker/]()
 

Current latest version:

```
> docker pull grobid/grobid:0.7.2
```

* Run the container:

```
> docker run --rm --gpus all -p 8070:8070 grobid/grobid:0.7.2
```


In [ ]:
path = r'PDFs/*.pdf'
files = glob.glob(path)
print(files[0])

In [ ]:
GROBID_URL = 'http://localhost:8070'
# url = '%s/api/processHeaderDocument' % GROBID_URL 
# processFulltextDocument
url = '%s/api/processFulltextDocument' % GROBID_URL 
# fulltext  processFullText batch command will extract, structure and normalize in TEI the full text of pdf files. The needed parameters for that command are:
for pdf in files:
    p = pdf.split("/")[-1].split(".pdf")[0]
    pp = "../TEI/"+p+".xml"
    if not os.path.isfile(pp):
        xml = requests.post(url, files={'input': open(pdf, 'rb')}).text
        soup = BeautifulSoup(xml, "xml")
        s = soup.prettify()

        with open(pp, 'w') as f:
            f.write(s)
        print(pdf,"done.")
    else:
        print(pdf,"already done.")

In [3]:
def read_tei(tei_file):
    with open(tei_file, 'r') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup 

In [4]:
path = r'TEI/*.xml'
files = glob.glob(path)
FILE = files[0]
print(len(files))
print(FILE)

409
TEI\abideen_mitigation_2020.xml


In [5]:
allData = []
for FILE in files:
    ID = FILE.split(os.sep)[-1].split(".")[0]
    try:
        soup = read_tei(FILE)
        TITLE = soup.title.text.strip()
        ABSTRACT = soup.abstract.getText().strip()
        removeRef = [s.extract() for s in soup('ref')]
        removeBiblio = [s.extract() for s in soup('listBibl')]
        removeBiblio = [s.extract() for s in soup('back')]
        removeBiblio = [s.extract() for s in soup('figure')]

        txt = soup.find('body').find('text').getText(strip=True,separator="\n")

        allData.append([ID,TITLE,ABSTRACT,txt.strip()])
    except:
        print("Failed on",ID,"-",FILE)

c:\Users\JON80843\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Failed on li_role_2007 - TEI\li_role_2007.xml
Failed on lytle_predicted_2005 - TEI\lytle_predicted_2005.xml


In [12]:
lsdoi = glob.glob("cache_biblio/*.json")
lsdoi = [x.split(os.sep)[-1].split(".")[0] for x in lsdoi]
print(len(lsdoi))

382


We only keep articles with DOIs

In [14]:
df = pd.DataFrame(allData)
df.columns = ["article","title","abstract","text"]
df = df[df.article.isin(lsdoi)]
print(len(df))

327


And we save them

In [16]:
df.to_parquet("TEIs.parquet.gzip",compression="gzip")